In [ ]:
import os 
import importlib

from waffles.input_output.hdf5_structured import load_structured_waveformset
from waffles.data_classes.WaveformSet import WaveformSet
from waffles.data_classes.Waveform import Waveform
from waffles.np04_data.ProtoDUNE_HD_APA_maps import APA_map # apa map
from waffles.data_classes.ChannelWsGrid import ChannelWsGrid # channel grid

import integral_computation_function as integral_module
from integral_computation_function import channel_integral_computation

import waffles.np04_analysis.lightyield_vs_energy.scripts.utils_plots as utils_plots_module
from waffles.np04_analysis.lightyield_vs_energy.scripts.utils_plots import *


from waffles.np04_analysis.lightyield_vs_energy.scripts.MyAnaPeak import MyAnaPeak

In [ ]:
run_folder = "/afs/cern.ch/work/a/anbalbon/public/reading_rucio/run027361/full_streaming/no_selection"
fname = "processed_np04hd_raw_run027361_0001_dataflow0_datawriter_0_20240622T004415.hdf5_structured.hdf5"
filepath = os.path.join(run_folder, fname)
FS_wfset = load_structured_waveformset(filepath)  # load HDF5 structured waveform set
            

In [ ]:
plotting_overlap_wf_PEAK_NEW(FS_wfset, n_wf= 20, show  = True, peak_bool = False, peak_beam_bool = False)

In [ ]:
importlib.reload(integral_module)
from integral_computation_function import channel_integral_computation

importlib.reload(utils_plots_module)
from waffles.np04_analysis.lightyield_vs_energy.scripts.utils_plots import *

apa = 1
endpoint = 104
channel = 7
integration_analysis_label = 'integrator'
period = 'june'
grid_apa_FS = ChannelWsGrid(APA_map[apa], FS_wfset, compute_calib_histo=False)
# for endpoint in grid_apa_FS.ch_wf_sets.keys():
#     for channel in grid_apa_FS.ch_wf_sets[endpoint].keys():
#         grid_apa_FS.ch_wf_sets[endpoint][channel] = channel_integral_computation(grid_apa_FS.ch_wf_sets[endpoint][channel], period = period, integration_analysis_label = integration_analysis_label)

grid_apa_FS.ch_wf_sets[endpoint][channel] = channel_integral_computation(grid_apa_FS.ch_wf_sets[endpoint][channel], period = period, integration_analysis_label = integration_analysis_label, beam_average_timetick = 15600)

# plotting_overlap_wf_(grid_apa.ch_wf_sets[endpoint][channel], n_wf = 10, show = True, save = False, x_min=None, x_max=None, y_min=None, y_max=None, int_ll=grid_apa.ch_wf_sets[endpoint][channel].waveforms[0].analyses[integration_analysis_label].result['integration_limits'][0], int_ul=grid_apa.ch_wf_sets[endpoint][channel].waveforms[0].analyses[integration_analysis_label].result['integration_limits'][1], baseline=None, output_folder = 'output', deconvolution = False, analysis_label = integration_analysis_label)
plotting_overlap_wf_PEAK_NEW(grid_apa_FS.ch_wf_sets[endpoint][channel], n_wf= 20, show  = True, peak_bool = True, peak_beam_bool = True, analysis_label=integration_analysis_label, int_ll=grid_apa_FS.ch_wf_sets[endpoint][channel].waveforms[0].analyses[integration_analysis_label].result['integration_limits'][0], int_ul=grid_apa_FS.ch_wf_sets[endpoint][channel].waveforms[0].analyses[integration_analysis_label].result['integration_limits'][1])

In [ ]:
analysis_label = 'test_peak_finding'
analysis_class= MyAnaPeak


ip = IPDict(baseline_limits= [0, 50],
            int_ll = 55,
            int_ul = 100,
            amp_ll = 55,
            amp_ul= 100,
            invert = False,
            baseline_method = "EasyMedian"
            )

_ = FS_wfset.analyse(label=analysis_label ,analysis_class=analysis_class, input_parameters=ip, checks_kwargs = IPDict({'points_no': FS_wfset.points_per_wf}), overwrite=True)

print('done\n\n')

In [ ]:
plotting_overlap_wf_PEAK_NEW(FS_wfset, n_wf= 20, show  = True, save = False, peak_bool = True, peak_beam_bool = True, analysis_label=analysis_label)

In [ ]:
FS_offset_list = []
FS_beam_peak_absolute_time = []
for wf in FS_wfset.waveforms:
    FS_offset_list.extend(wf.analyses[analysis_label].result['time_offset'])
    FS_beam_peak_absolute_time.extend(wf.analyses[analysis_label].result['beam_peak_absolute_time'])

fig = go.Figure(
    data=[
        go.Histogram(
            x=FS_offset_list,
            nbinsx=20000,
            marker_color='royalblue',
            opacity=0.7
        )
    ]
)

fig.show()

In [ ]:
# Filtro per selezionare solo i beam events (sia FS che ST)
def beam_filter(waveform: Waveform, analysis_label) -> bool:
    if len(waveform.analyses[analysis_label].result['beam_peak_index'])>0:
        return True
    else:
        return False

In [ ]:
FS_beam_wfset = WaveformSet.from_filtered_WaveformSet(FS_wfset, beam_filter, analysis_label)
print('FS events: ', len(FS_wfset.waveforms))
print('FS beam events:', len(FS_beam_wfset.waveforms))

In [ ]:
plotting_overlap_wf_PEAK_NEW(FS_beam_wfset, n_wf= 20, show  = True, save = False, peak_bool = True, peak_beam_bool = True, analysis_label=analysis_label)

In [ ]:
importlib.reload(integral_module)
from integral_computation_function import channel_integral_computation

apa = 1
endpoint = 104
channel = 7
integration_analysis_label = 'integrator'
period = 'june'
grid_apa_FS = ChannelWsGrid(APA_map[apa], FS_beam_wfset, compute_calib_histo=False)
# for endpoint in grid_apa_FS.ch_wf_sets.keys():
#     for channel in grid_apa_FS.ch_wf_sets[endpoint].keys():
#         grid_apa_FS.ch_wf_sets[endpoint][channel] = channel_integral_computation(grid_apa_FS.ch_wf_sets[endpoint][channel], period = period, integration_analysis_label = integration_analysis_label)

grid_apa_FS.ch_wf_sets[endpoint][channel] = channel_integral_computation(grid_apa_FS.ch_wf_sets[endpoint][channel], period = period, integration_analysis_label = integration_analysis_label)

# plotting_overlap_wf_(grid_apa.ch_wf_sets[endpoint][channel], n_wf = 10, show = True, save = False, x_min=None, x_max=None, y_min=None, y_max=None, int_ll=grid_apa.ch_wf_sets[endpoint][channel].waveforms[0].analyses[integration_analysis_label].result['integration_limits'][0], int_ul=grid_apa.ch_wf_sets[endpoint][channel].waveforms[0].analyses[integration_analysis_label].result['integration_limits'][1], baseline=None, output_folder = 'output', deconvolution = False, analysis_label = integration_analysis_label)
plotting_overlap_wf_PEAK_NEW(grid_apa_FS.ch_wf_sets[endpoint][channel], n_wf= 5, show  = True, peak_bool = True, peak_beam_bool = True, analysis_label=analysis_label, int_ll=grid_apa_FS.ch_wf_sets[endpoint][channel].waveforms[0].analyses[integration_analysis_label].result['integration_limits'][0], int_ul=grid_apa_FS.ch_wf_sets[endpoint][channel].waveforms[0].analyses[integration_analysis_label].result['integration_limits'][1])